# Pipeline With Transformers

In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

In [ ]:
df = pd.read_csv('titanic_data.csv')

In [ ]:
df.sample(3)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
624,625,0,3,"Bowen, Mr. David John ""Dai""",male,21.0,0,0,54636,16.1000,NaN,S
420,421,0,3,"Gheorgheff, Mr. Stanio",male,NaN,0,0,349254,7.8958,NaN,C
286,287,1,3,"de Mulder, Mr. Theodore",male,30.0,0,0,345774,9.5000,NaN,S


In [ ]:
df.drop(columns = ['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace = True)

In [ ]:
x = df.drop(columns = ['Survived'])
y = df['Survived']

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state= 12)

# Create Transformers
1. Impute Transformer

In [ ]:
trf1 = ColumnTransformer([
    ('impute_age', SimpleImputer(), [2]),
    ('impute_embarked', SimpleImputer(strategy = 'most_frequent'), [6])],
    remainder='passthrough')

2. OneHotEncoder Transformer

In [ ]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked', OneHotEncoder(handle_unknown= 'ignore'),[1,3])],
    remainder='passthrough')

3. Scaling Transformer

In [ ]:
trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(), slice(0,10))],
    remainder='passthrough')


4. Feature_Selection Transformer

In [ ]:
trf4 = SelectKBest(score_func= chi2, k = 8)

5. To Train Model

In [ ]:
trf5 = DecisionTreeClassifier()

# Create Pipeline->

In [ ]:
pipe = Pipeline([
    ('trf1', trf1),
    ('trf2', trf2),
    ('trf3', trf3),
    ('trf4', trf4),
    ('trf5', trf5)])

In [ ]:
pipe.fit(x_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  [1, 3])])),
                ('trf3',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x7deca6025300>)),
                ('trf5', DecisionTreeClassifier())])

In [ ]:
from sklearn import set_config
set_config(display = 'diagram') #to get the workflow of the diagram.

---Use the pipeline---

In [ ]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore'),
                                  [1, 3])]),
 'trf3': ColumnTransformer(remainder='passthrough',
                   transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x7deca6025300>),
 'trf5': DecisionTreeClassifier()}

In [ ]:
pipe.named_steps['trf3']

ColumnTransformer(remainder='passthrough',
                  transformers=[('scale', MinMaxScaler(), slice(0, 10, None))])

In [ ]:
# chechk trf3's functions
pipe.named_steps['trf3'].transformers_[0][1]

MinMaxScaler()

In [ ]:
y_pred = pipe.predict(x_test)

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.770949720670391

# Cross Validation using pipeline

In [ ]:
from sklearn.model_selection import cross_val_score
cross_val_score(pipe, x_train, y_train, cv = 5, scoring = 'accuracy').mean()

np.float64(0.7976854131783709)

# GridSearch using pipeline (HyperParameter)

In [ ]:
prmt = {'trf5__max_depth' : [1, 2, 3, 4, 5, None]}

In [ ]:
from sklearn.model_selection import GridSearchCV

grid = GridSearchCV(pipe, prmt, cv=5, scoring = 'accuracy')

grid.fit(x_train, y_train)


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore'),
                                                                         [1,
                                                                          3])])),
                                       ('trf3',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x7deca6025300>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [ ]:
grid.best_score_

np.float64(0.825874125874126)

In [ ]:
grid.best_params_

{'trf5__max_depth': 3}

# Export Whole pipeline

In [ ]:
import pickle
pickle.dump(pipe, open('pipe.pkl', 'wb'))